# Purpose of the Notebook

# Problem Statement

# Module Import

In [21]:
import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
#os.environ["CUDA_VISIBLE_DEVICES"] = ""
import datetime
from collections import Counter
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re

#to speed up pandas operands
from pandarallel import pandarallel

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import wordnet
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.layers import Dense, Dropout, Bidirectional
from keras.layers import GlobalAveragePooling1D, Flatten, Conv1D
from keras.layers import MaxPooling1D, GlobalMaxPooling1D

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.optimizers import Adam
from keras import regularizers

import keras.utils as ku 


from project5_utils import my_print
from project5_utils import my_wait

from MyMovieReview import MovieReview
from MyMovieReview import MovieReviewGenerator

from mymovie import Movie

In [22]:
%matplotlib inline 

In [ ]:
nltk.download('stopwords')

In [23]:
LOG_FILE = open("../Data/notebook.log","w")
DEBUG = 1

In [24]:
#Setting these for now will adjust them
VOCAB_SIZE = 400000
EMBEDDING_DIM = 300

#will calc this based on training set
#MAX_LENGTH = 1000
TRUC_TYPE = "post"
PAD_TYPE = "post"
#Out of Value Token -- or token to indicate a 
OOV_TOK = "<OOV>"

TRAIN_SET_PERCENT =  90.0
EPOCHS = 100
RANDOM_STATE = 42

BATCH_SIZE = 96

# Data Processing
Get reviews from PKL files into format that can be tokenized

In [25]:
MOVIE_REVIEWS_PKL_FILE = open("../Data/Reviews_back7.pkl","rb")

list_of_movie_reviews = []

while 1:
    try:
        temp_review = pickle.load(MOVIE_REVIEWS_PKL_FILE)
        list_of_movie_reviews.append(temp_review)
        my_print("added this review to the list ==>\n{}\n".format(temp_review), DEBUG, LOG_FILE)
    except EOFError:
        my_print("DONE reading movie reviews", DEBUG, LOG_FILE)
        break
        
MOVIE_REVIEWS_PKL_FILE.close()
print("Number of movie reviews read: {}".format(len(list_of_movie_reviews)))

Number of movie reviews read: 35404


In [26]:
col_names = list(list_of_movie_reviews[0].__dict__.keys())
#col_values = list(list_of_movie_reviews[0].__dict__.values()) --> TODO: REMOVE

In [27]:
list_of_movie_review_lists = []

for review_col_name in col_names:
    list_of_movie_review_data_vals = []
    
    for review_obj in list_of_movie_reviews:
        list_of_movie_review_data_vals.append ( review_obj.__dict__[review_col_name] )
        
    list_of_movie_review_lists.append(list_of_movie_review_data_vals)
    
reviews_dict = dict(zip(col_names,list_of_movie_review_lists))
reviews_df = pd.DataFrame(reviews_dict)
reviews_df.head()

,review_text,review_star_rating,reviewer_name,directlink_url,reviewlink_url,title,review_title,review_date
0,At the end of this movie there's a shot of the...,2,veardleyw,http://www.imdb.com/title/tt4669788/,http://www.imdb.com/review/rw4675276/,On the Basis of Sex,"Feeble, telemovie-style travesty of a potentia...",20 February 2019
1,The film starts in 1956 with RBG (Felicity Jon...,8,nogodnomasters,http://www.imdb.com/title/tt4669788/,http://www.imdb.com/review/rw4783407/,On the Basis of Sex,It's not a movement if everyone is just sitting.,15 April 2019
2,This well meaning movie shows the discriminati...,5,phd_travel,http://www.imdb.com/title/tt4669788/,http://www.imdb.com/review/rw4590175/,On the Basis of Sex,Well intentioned but miscast,18 January 2019
3,Why would you make a movie about the second fe...,1,Viking131313,http://www.imdb.com/title/tt4669788/,http://www.imdb.com/review/rw4621289/,On the Basis of Sex,Makes no sense.,2 February 2019
4,This movie was very well done. Good acting and...,10,MikeChm,http://www.imdb.com/title/tt4669788/,http://www.imdb.com/review/rw4583636/,On the Basis of Sex,Well worth going to see!,15 January 2019


**QUICK AND DIRTY CLEANING**

TODO: Go back and Improve it

In [28]:
#Get rid of instances of people spamming the same review
reviews_df.drop_duplicates(subset=["review_text"],keep="first",inplace=True)

In [29]:
#Get rid of reviews where "review_star_rating" is missing or "review_text" is missing
mask = (reviews_df["review_text"] == "EMPTY") | (reviews_df["review_star_rating"] == -1)
mask_keep = ~mask

In [30]:
reviews_df = reviews_df[mask_keep]

In [31]:
reviews_df.dropna(inplace=True)

In [32]:
reviews_df.shape

(35342, 8)

In [33]:
text_of_reviews = reviews_df[["review_title","review_text","review_star_rating"]]

Text corpus will be composed of the review title concatenated with the review text

In [34]:
text_of_reviews["review_observation"] = text_of_reviews["review_title"] + " " + text_of_reviews["review_text"]

<ipython-input-34-b4d769d80be9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_of_reviews["review_observation"] = text_of_reviews["review_title"] + " " + text_of_reviews["review_text"]


In [35]:
text_of_reviews = text_of_reviews.drop(["review_title","review_text"], axis=1)

In [36]:
text_of_reviews = text_of_reviews.rename(columns= {"review_observation":"review_text"})

In [37]:
text_of_reviews.head()

,review_star_rating,review_text
0,2,"Feeble, telemovie-style travesty of a potentia..."
1,8,It's not a movement if everyone is just sittin...
2,5,Well intentioned but miscast This well meaning...
3,1,Makes no sense. Why would you make a movie abo...
4,10,Well worth going to see! This movie was very w...


**TODO:** Do Some stats

* Are the classes balanced?
* What's the mean, mode, stddev of the classes?
* Length of reviews: mean, mode, std dev, and all of this for each class

In [38]:
len(text_of_reviews)

35342

## Cleaning and Lemmatization
To reduce dimentionality of the corpus


In [39]:
def remove_html_punct(row):
    """This function removes HTML and punctuation and anything that is or contains a number from the Text"""
    
    NLTK_WORDS = set(nltk.corpus.words.words())
    
    text_to_process = row["review_text"]
    text_to_process = text_to_process.lower()
    text_to_process = re.sub("<.*?>","",text_to_process)
    text_to_process = re.sub("[\.|\!|\?|\,|\;|\:|\&|\(|\)|\-|\%|_|\#|\$|\*|\+|\/|\=|\[|\]|\^|\`|\{|\}|\~]","",text_to_process)
    text_to_process = re.sub("\w*\d+\w*","",text_to_process)
    text_to_process_list = text_to_process.split()
    text_to_process_list = [review_word for review_word in text_to_process_list if review_word in NLTK_WORDS]
    text_to_process = " ".join(text_to_process_list)
    return text_to_process

In [40]:
pandarallel.initialize()
start_time = datetime.datetime.now()

text_of_reviews["review_text"] = text_of_reviews.parallel_apply(remove_html_punct, axis=1)

end_time = datetime.datetime.now()
print("time spent doing operation is {}".format(end_time - start_time))

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
time spent doing operation is 0:06:37.201333


In [41]:
text_of_reviews["review_text"] = text_of_reviews["review_text"].apply(lambda x : x.lower())

In [42]:
def tokenize_text(row):
    temp_tokenized_txt = word_tokenize(row["review_text"])
    return temp_tokenized_txt


In [43]:
pandarallel.initialize()
start_time = datetime.datetime.now()

text_of_reviews["review_text"] = text_of_reviews.parallel_apply(tokenize_text,axis=1)

end_time = datetime.datetime.now()
print("time spent doing operation is {}".format(end_time - start_time))

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
time spent doing operation is 0:00:03.131240


In [44]:
wordNetLemmatizer = WordNetLemmatizer()

In [45]:
def calc_partofspeech(raw_pos):
    """translates from POS generated by pos_tag() to a POS encoding that WordNetLemmatizer.lemmatize() understands"""
    #print("word = ",word)
    #[(output_word, output_pos)] = pos_tag(word_tokenize(word))
    
    #decode output_pos to the pos required by the lemmatizer
    
    if "JJ" in raw_pos:
        pos = "a"
    elif "RB" in raw_pos:
        pos = "r"
    elif "VB" in raw_pos:
        pos = "v"
    else:
        pos = "n"
    
    return pos

In [46]:
def lemmatize_it(row):
    
    pos_result = pos_tag(row["review_text"])
    
    temp_mydoc_lemmatized = []
    
    for myword,myPOS in pos_result:
        temp_word_lemmatized = wordNetLemmatizer.lemmatize(myword,calc_partofspeech(myPOS))
        temp_mydoc_lemmatized.append(temp_word_lemmatized)
    
    return temp_mydoc_lemmatized
        


In [47]:
pandarallel.initialize()
start_time = datetime.datetime.now()

text_of_reviews["review_text"] = text_of_reviews.parallel_apply(lemmatize_it,axis=1)

end_time = datetime.datetime.now()
print("time spent doing operation is {}".format(end_time - start_time))

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
time spent doing operation is 0:00:33.020713


In [48]:
pandarallel.initialize()
start_time = datetime.datetime.now()
text_of_reviews["review_text"]  = text_of_reviews.parallel_apply(lambda x : " ".join(x["review_text"]), axis=1)
end_time = datetime.datetime.now()
print("time spent doing operation is {}".format(end_time - start_time))

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
time spent doing operation is 0:00:02.877137


### Stop word removal
Should I do it?

It lowers dimensionality which can improve training time.

But for a sequence model won't removing the stop words hurt the ability of the model to learn sequences that may be indicative of a specific class? In this project I need pretty high precision because I have 10 categories to classify based on text.

Some stuff I've read:

**Pro or Con**:

* https://stackoverflow.com/questions/37325914/should-i-remove-stopwords-when-feed-sentence-to-rnn
* https://www.quora.com/Is-text-preprocessing-e-g-stop-words-removing-still-necessary-in-neural-based-NLP
* https://www.quora.com/Is-text-preprocessing-e-g-stop-words-removing-still-necessary-in-neural-based-NLP

Reading the last link I think I should keep the stop words. The star rating applied to a reviews is a form of sentiment about the review. If a reviewer says "I was not happy with the ending of the movie," stop word removal may convert that to "happy movie ending". This may result in two reviews containing the words "happy movie ending": One with a review of 2 and another with a review of 8 with the original text for the latter score being "I was happy with the ending of the movie."

So will not do stop word removal for now but will keep in as an option if results seem to call for it.


In [49]:
text_of_reviews.head()

,review_star_rating,review_text
0,2,feeble travesty of a potentially fascinating s...
1,8,not a movement if everyone be just sit the fil...
2,5,well intentioned but miscast this well mean mo...
3,1,no sense why would you make a movie about the ...
4,10,well worth go to see this movie be very well d...


In [50]:
#shifting range of ratings from 1,10 to 0,9
text_of_reviews["review_star_rating"] = text_of_reviews["review_star_rating"] - 1

In [51]:
text_of_reviews.shape

(35342, 2)

In [52]:
#TODO Randomize sets or find tensorflow train test split or use sklearn train test split
train_set_size = int ( ( text_of_reviews.shape[0] * TRAIN_SET_PERCENT ) / 100 )
#test_set_size =  text_of_reviews.shape[0] - train_set_size

review_indices = np.arange(text_of_reviews.shape[0]) 

np.random.shuffle( review_indices)


text_of_reviews = text_of_reviews.iloc[review_indices]

training_indices = review_indices[:train_set_size]
testing_indices = review_indices[train_set_size:]


print(train_set_size)
print(text_of_reviews.shape)
print(training_indices.shape)
print(testing_indices.shape)


31807
(35342, 2)
(31807,)
(3535,)


In [53]:
text_of_reviews.head()

,review_star_rating,review_text
25248,5,this a bad movie but not brilliant triple be a...
2104,7,best star film by out of all the the i think t...
26159,0,total garbage i be a fan of the show i it but ...
8704,0,waste your time the bad piece of dirt this yea...
30075,6,a good comedy with a great cast a good old fas...


In [54]:
#spent hours trying to re-implement train_test_split() :*-(

#train_new_reviews = text_of_reviews.loc[text_of_reviews.index.intersection(training_indices),"review_text"]
#train_new_reviews_rating = text_of_reviews.loc[text_of_reviews.index.intersection(training_indices),"review_star_rating"]

#train_text_reviews = text_of_reviews.sample(n=train_set_size, random_state=RANDOM_STATE, )

#test_new_reviews = text_of_reviews.loc[text_of_reviews.index.intersection(testing_indices),"review_text"]
#test_new_reviews_rating = text_of_reviews.loc[text_of_reviews.index.intersection(testing_indices),"review_star_rating"]

X = text_of_reviews["review_text"]
y = text_of_reviews["review_star_rating"]

Test_Size = 1.0 - (TRAIN_SET_PERCENT/100.0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=Test_Size, random_state=RANDOM_STATE)


In [55]:
#TODO
#Lemmatization
#stop word removal
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(31807,)
(31807,)
(3535,)
(3535,)


In [56]:
Max_Length = max([len(doc) for doc in X_train])
print("Max_Length = {}".format(Max_Length))

Max_Length = 7779


In [57]:
tokenizer = Tokenizer(oov_token=OOV_TOK, num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(X_train)
total_words = len(tokenizer.word_index)+1
i = 0
for key, value in tokenizer.word_index.items():
    
    if i == 10:
        break
        
    print("word_index[{}] = {}".format(key, value))
    i += 1
    
print("\ntotal words is {}".format(total_words))

word_index[<OOV>] = 1
word_index[the] = 2
word_index[be] = 3
word_index[a] = 4
word_index[and] = 5
word_index[to] = 6
word_index[of] = 7
word_index[it] = 8
word_index[in] = 9
word_index[i] = 10

total words is 25515


In [58]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_seq_pad = pad_sequences(X_train_seq, maxlen=Max_Length, padding=PAD_TYPE, truncating=TRUC_TYPE)

In [59]:
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_seq_pad = pad_sequences(X_test_seq, maxlen=Max_Length, padding=PAD_TYPE, truncating=TRUC_TYPE)

In [60]:
for i, thing in enumerate(X_train_seq):
    if i == 3:
        break
    print("thing = {}".format(thing))

thing = [29, 51, 69, 42, 2793, 72, 12, 2, 2286, 15, 12, 17, 3, 596, 52, 212, 14, 27, 7, 2, 103, 97, 509, 294, 48, 19, 2, 103, 9, 870, 411, 30, 10, 43, 74, 12, 17, 14, 4, 111, 7062, 5, 4, 114, 7, 3147, 10, 64, 547, 82, 11, 10, 3, 19, 683, 12, 3, 4, 212, 6690, 2467, 2793, 140, 628, 8781, 1617, 97, 13, 12, 17, 65, 19, 18, 3, 46, 7, 242, 9, 2, 16, 6, 635, 74, 2, 1626, 7, 870, 97, 304, 8, 3, 482, 69, 42, 1159, 5, 1744, 72, 83, 531, 172, 97, 16, 12, 6, 2, 17, 5, 4, 1904, 7, 1300, 306, 8, 48, 20, 43, 186, 319, 85, 8, 43, 6, 3, 163, 12, 17, 23, 44, 8, 6, 23, 8, 1215, 6, 2299, 628, 5, 473, 15, 2, 1210, 7, 26, 373, 13554, 16, 79, 1142, 486, 98, 25, 83, 7, 2, 85, 37, 63, 54, 12, 17, 448, 58, 54, 2, 383, 97, 7, 2, 5, 5, 3, 149, 155, 15, 4, 171, 7, 106, 3, 19, 11, 153, 16, 8, 179, 6, 3, 669, 3, 8, 6, 3, 2, 138, 43, 169, 82, 3, 354, 44, 93, 18, 3, 4, 973, 7, 8537, 84, 58, 3, 4, 553, 439, 7, 104, 2825, 3, 7587, 5, 180, 216, 89, 987, 3, 57, 620, 27, 6193, 64, 90, 286, 4, 128, 3662, 21, 2, 123, 15, 51,

In [61]:
print(X_train_seq_pad.shape)

(31807, 7779)


In [62]:
#Convert to numpy arrays to work with Tensorflow
X_train_seq_pad = np.array(X_train_seq_pad)
y_train = np.array(y_train)

X_test_seq_pad = np.array(X_test_seq_pad)
y_test = np.array(y_test)



In [63]:
X_train_seq_pad.shape, y_train.shape

((31807, 7779), (31807,))

In [64]:
X_test_seq_pad.shape, y_test.shape

((3535, 7779), (3535,))

In [65]:
X_test_seq_pad[:3]

array([[   4,  445,  427, ...,    0,    0,    0],
       [ 916,  545, 2785, ...,    0,    0,    0],
       [  52,   69,  416, ...,    0,    0,    0]], dtype=int32)

In [66]:
y_train[:3]

array([7, 9, 5])

# Baseline Model
The base line model is a custom word embedding providing vectorized words to two layers of 1-D convolutional next networks. According to Chollet a 1-D CNN model can (if properly tuned) do well at text classification.

But the limitation is that if information in the text is highly dependend on the sequence of words, it will be limited. I think because sentiment in English can be highly determined not only by the words used but by the order in which they're used this model won't score well, but is a baseline of how well a simpler-to-train option could do.

TODO: add Reference quoting Chollet

In [70]:
#baseline_model = Sequential([
#    Embedding(VOCAB_SIZE + 1,EMBEDDING_DIM, input_length=MAX_LENGTH),
#    GlobalAveragePooling1D(),
#    Dense(500, activation="relu"),
#    Dense(250, activation="relu"),
#    Dense(125, activation="relu"),
#    Dense(60, activation="relu"),
#    Dense(30, activation="relu"),
#    Dense(15, activation="relu"),
#    Dense(10, activation="softmax")
#])

baseline_model = Sequential()
baseline_model.add(Embedding(VOCAB_SIZE+1, EMBEDDING_DIM, input_length=Max_Length))
baseline_model.add(Conv1D(32,7,activation="relu"))
baseline_model.add(MaxPooling1D(5))
baseline_model.add(Conv1D(32,7,activation="relu"))
baseline_model.add(GlobalMaxPooling1D())
baseline_model.add(Dense(15, activation="relu", kernel_regularizer=regularizers.l2(0.001)))
baseline_model.add(Dropout(0.2))
baseline_model.add(Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(0.001)))

baseline_model.compile(optimizer="adadelta", loss="sparse_categorical_crossentropy", metrics=["acc"])
baseline_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7779, 64)          1216064   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 7773, 32)          14368     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1554, 32)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1548, 32)          7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 15)                495       
_________________________________________________________________
dropout_1 (Dropout)          (None, 15)               

In [73]:
call_back_list = [
    keras.callbacks.ModelCheckpoint(
        filepath = "../Data/baseline_model.h5",
        monitor = "val_acc",
        save_best_only = True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor = "val_acc",
        factor = 0.1,
        patience = 3
    ),
    keras.callbacks.EarlyStopping(patience=15, verbose=1,restore_best_weights=True)
    
    
]

In [74]:
training_history = baseline_model.fit(X_train_seq_pad,
                                      y_train,
                                      epochs=EPOCHS,
                                      batch_size=BATCH_SIZE,
                                      callbacks=call_back_list,
                                      validation_split=Test_Size, 
                                      verbose=1)

Epoch 1/100
299/299 [==============================] - 28s 94ms/step - loss: 2.3245 - acc: 0.1056 - val_loss: 2.3229 - val_acc: 0.1179 - lr: 0.0010
Epoch 2/100
299/299 [==============================] - 28s 94ms/step - loss: 2.3232 - acc: 0.1074 - val_loss: 2.3220 - val_acc: 0.1195 - lr: 0.0010
Epoch 3/100
299/299 [==============================] - 28s 94ms/step - loss: 2.3228 - acc: 0.1089 - val_loss: 2.3211 - val_acc: 0.1220 - lr: 0.0010
Epoch 4/100
299/299 [==============================] - 28s 94ms/step - loss: 2.3221 - acc: 0.1115 - val_loss: 2.3202 - val_acc: 0.1254 - lr: 0.0010
Epoch 5/100
299/299 [==============================] - 28s 94ms/step - loss: 2.3208 - acc: 0.1134 - val_loss: 2.3193 - val_acc: 0.1267 - lr: 0.0010
Epoch 6/100
299/299 [==============================] - 28s 94ms/step - loss: 2.3198 - acc: 0.1177 - val_loss: 2.3184 - val_acc: 0.1298 - lr: 0.0010
Epoch 7/100
299/299 [==============================] - 28s 93ms/step - loss: 2.3189 - acc: 0.1205 - val_loss: 2.

KeyboardInterrupt: 

In [ ]:
loss = training_history.history["loss"]
val_loss = training_history.history["val_loss"]
acc = training_history.history["acc"]
val_acc = training_history.history["val_acc"]
epochs = range(1,len(loss)+1)

plt.figure(figsize=(20,20))
plt.plot(epochs, loss,"bo",label="Training Loss")
plt.plot(epochs, val_loss,"b",label="Validation Loss")

plt.legend()
#plt.show()
plt.savefig("../Documents/Images/loss_curves.png",dpi=400)

In [ ]:
plt.figure(figsize=(20,20))
plt.plot(epochs, acc,"go",label="Training Acc")
plt.plot(epochs, val_acc,"g",label="Validation Acc")
plt.legend()
#plt.show()
plt.savefig("../Documents/Images/accuracy_curves.png",dpi=400)

Huge overfit!

Things I need to try:

* Collect more data
* Use a pretrained word embedding
* Regularize
* Add callbacks to make training work harder for me
* which should I use as a loss function: `sparse_categorical_crossentropy` or `categorical_crossentropy`
* Which optimizer should I use: `adam` or `RMSprop(lr=0.1)`

**TODO**:
Find out what it means when val_loss improves by val_acc does not improve.


## Test Example

In [ ]:
score, acc = baseline_model.evaluate(X_test_seq_pad, y_test, batch_size=BATCH_SIZE)

In [ ]:
print(score)

In [ ]:
print(acc)

In [ ]:
#TODO Add F1, 

# Updated Model
Will have a model with the following features:

* Pretrained word embedding
* LSTM (or GRU)
* Possibliy bidirectional
* More regularization
* If training is too long: Opt for GRU. Also may opt for a CNN+RNN hybrid


### Pretrained Word Embedding

In [75]:
!./get_word_vectors.sh


--2020-12-03 16:58:27--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-03 16:58:27--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-03 16:58:27--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘../Data/glove.6B.zip’


In [80]:
#build embedding matrix

word_index = tokenizer.word_index

embedding_dict = {}

GLOVE6B300_FILE = open("../Data/glove.6B.300d.txt","r")

for line in GLOVE6B300_FILE:
    line_values = line.split(" ")
    word = line_values[0]
    vector_components = np.asarray(line_values[1:], dtype="float32")
    embedding_dict[word] = vector_components

Embedding_Matrix = np.zeros((VOCAB_SIZE + 1, EMBEDDING_DIM))

for word, index_value in word_index.items():
    embedding_vector = embedding_dict.get(word)
    
    if embedding_vector is not None:
        Embedding_Matrix[index_value] = embedding_vector



ValueError: could not broadcast input array from shape (300) into shape (64)

# Conclusion